In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Define functions

In [2]:
def visualize_point_cloud(nome_point_cloud):
    pcd = o3d.io.read_point_cloud(nome_point_cloud, format = 'ply')
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])
    return pcd

In [3]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert = True)

    #print("Showing outliers (red) and inliers (gray): ")
    #outlier_cloud.paint_uniform_color([1, 0, 0])
    #inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud])
    #print("inlier_cloud: ", inlier_cloud)

In [4]:
def save_point_cloud(nome_file, punti_utili, colori_utili):
    file = open(nome_file,"w")
    file.write("ply \n")
    file.write("format ascii 1.0 \n")
    file.write("element vertex " + str(punti_utili.shape[0]) + "\n")
    file.write("property float x \n")
    file.write("property float y \n")
    file.write("property float z \n")
    file.write("property uchar red \n")
    file.write("property uchar green \n")
    file.write("property uchar blue \n")
    file.write("end_header \n")
    i = 0
    j = 0
    x = 0
    for i in range(np.shape(punti_utili)[0]):
        for j in range (3):
            file.write(str(punti_utili[i, j]) + " ")
        for x in range (3):
            file.write(str(colori_utili[i, x]) + " ")
        file.write("\n")

In [5]:
#Modificare il valore di nb_neighbors per specificare quanti vicini devono essere presi in considerazione per calcolare la distanza media per un dato punto
#Modificare il valore di std_ratio in base a quanto aggressiva si vuole la funzione, rispettivamente più è basso il valore più outlier verranno eliminati 
def statistical_outlier_removal(pcd, nb_neighbors, std_ratio):
    print("Statistical outlier removal")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors, std_ratio)
    display_inlier_outlier(pcd, ind)
    return cl

In [6]:
#Modificare il valore di nb_points per selezionare la quantità minima di punti che la sfera deve contenere
#Modificare il valore di radius per definire il raggio della sfera che verrà utilizzata per il conteggio dei vicini
def radius_outlier_removal(pcd, nb_points, radius):
    print("Radius outlier removal")
    cl, ind = pcd.remove_radius_outlier(nb_points, radius)
    display_inlier_outlier(pcd, ind)
    return cl

# Point cloud cleaning

In [25]:
pcd = visualize_point_cloud("PointCloud2000Buono.ply")
#pcd = visualize_point_cloud("PointCloud2000Fallato.ply")
#pcd = visualize_point_cloud("pointCloudPezzoBuono.ply")
#pcd = visualize_point_cloud("pointCloudPezzoFallato.ply")

[[ 5.97263  -1.9696    1.1652  ]
 [ 5.5911   -8.48562   4.05852 ]
 [-6.51422  -1.06304   1.56652 ]
 ...
 [ 3.55452  -6.54829   5.93991 ]
 [ 1.28527   0.940656  6.68124 ]
 [ 1.21416   0.310779  6.97567 ]]


In [8]:
red_value = 0.88
green_value = 1
#filtra i colori più vicini al rosso ed elimina le telecamere che sono completamente verdi
filtered_pcd = np.where((np.asarray(pcd.colors)[:,0] < red_value) & (np.asarray(pcd.colors)[:,1] != green_value))
print(filtered_pcd)
print(np.shape(filtered_pcd))

(array([ 3618,  3619,  3620, ..., 89181, 89182, 89183], dtype=int64),)
(1, 84150)


In [9]:
punti_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
colori_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
print(np.shape(punti_utili))
print(np.shape(colori_utili))

(84150, 3)
(84150, 3)


In [10]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if (np.asarray(pcd.colors)[i,0] < red_value) & (np.asarray(pcd.colors)[i,1] != green_value):
        for j in range(3):
            punti_utili[x,j] = np.asarray(pcd.points)[i,j]
        x = x + 1
print(punti_utili)

[[ 0.596226 -1.69364   7.66143 ]
 [ 1.19864  -1.4703    6.6767  ]
 [-0.652911 -1.15012   6.82994 ]
 ...
 [ 2.75244  -6.32066   6.12035 ]
 [ 1.33186  -1.50067   6.70713 ]
 [ 1.06321  -0.981919  7.31075 ]]


In [11]:
#crea una nuova matrice con i colori dei punti che non superano la soglia e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if (np.asarray(pcd.colors)[i,0] < red_value) & (np.asarray(pcd.colors)[i,1] != green_value):
        for j in range(3):
            colori_utili[x,j] = np.asarray(pcd.colors)[i,j] * 255
        x = x + 1
colori_utili = colori_utili.astype(int)
print(colori_utili)

[[158 127  96]
 [150 166 187]
 [120  76  66]
 ...
 [146  80  52]
 [207 198 197]
 [159 147 193]]


In [12]:
save_point_cloud("pointCloud_PrimaModifica.ply", punti_utili, colori_utili)

In [22]:
pcd_first_modify = visualize_point_cloud("pointCloud_PrimaModifica.ply")

[[ 5.68844e-02  7.59518e-01  6.17800e+00]
 [-9.34126e-01  1.50593e+00  6.13586e+00]
 [ 3.87531e-01  1.41671e+00  5.85540e+00]
 ...
 [-1.46257e-01  8.32682e-01  5.81137e+00]
 [-1.26488e+00  1.81604e+00  5.44585e+00]
 [-1.43104e+00  6.50741e-03  6.87100e+00]]


In [14]:
cl_statical = statistical_outlier_removal(pcd_first_modify, 20, 2.0) #Uguale per tutte e quattro le point cloud iniziali

Statistical outlier removal
Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [15]:
cl_radius = radius_outlier_removal(cl_statical, 16, 0.5) #Uguale per tutte e quattro le point cloud iniziali

Radius outlier removal
Showing outliers (red) and inliers (gray): 


In [16]:
#filtra i colori più scuri
filtered_cl_radius = np.where((np.asarray(cl_radius.colors)[:,0] > 0.20) & (np.asarray(cl_radius.colors)[:,1] > 0.20) &
                             (np.asarray(cl_radius.colors)[:,2] > 0.20))
print(filtered_cl_radius)
print(np.shape(filtered_cl_radius))

(array([    0,     1,     2, ..., 80118, 80119, 80120], dtype=int64),)
(1, 65411)


In [17]:
punti_utili_radius = np.zeros((np.shape(filtered_cl_radius)[1], 3), dtype = float)
colori_utili_radius = np.zeros((np.shape(filtered_cl_radius)[1], 3), dtype = float)
print(np.shape(punti_utili_radius))
print(np.shape(colori_utili_radius))

(65411, 3)
(65411, 3)


In [18]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(cl_radius.points))[0]):
    if ((np.asarray(cl_radius.colors)[i,0] > 0.20) & (np.asarray(cl_radius.colors)[i,1] > 0.20) & 
        (np.asarray(cl_radius.colors)[i,2] > 0.20)):
        for j in range(3):
            punti_utili_radius[x,j] = np.asarray(cl_radius.points)[i,j]
        x = x + 1
print(punti_utili_radius)

[[ 0.596226 -1.69364   7.66143 ]
 [ 1.19864  -1.4703    6.6767  ]
 [-0.652911 -1.15012   6.82994 ]
 ...
 [ 2.75244  -6.32066   6.12035 ]
 [ 1.33186  -1.50067   6.70713 ]
 [ 1.06321  -0.981919  7.31075 ]]


In [19]:
#creo una nuova matrice con i colori dei punti che non superano la soglia, e li riporto ai vecchi valori
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(cl_radius.points))[0]):
    if ((np.asarray(cl_radius.colors)[i,0] > 0.20) & (np.asarray(cl_radius.colors)[i,1] > 0.20) & 
        (np.asarray(cl_radius.colors)[i,2] > 0.20)):
        for j in range(3):
            colori_utili_radius[x,j] = np.asarray(cl_radius.colors)[i,j] * 255
        x = x + 1
colori_utili_radius = colori_utili_radius.astype(int)
print(colori_utili_radius)

[[158 127  96]
 [150 166 187]
 [120  76  66]
 ...
 [146  80  52]
 [207 198 197]
 [159 147 193]]


In [20]:
save_point_cloud("pointCloud_SecondaModifica.ply", punti_utili_radius, colori_utili_radius)

In [23]:
pcd_second_modify = visualize_point_cloud("pointCloud_SecondaModifica.ply")

[[ 5.68844e-02  7.59518e-01  6.17800e+00]
 [-9.34126e-01  1.50593e+00  6.13586e+00]
 [ 3.87531e-01  1.41671e+00  5.85540e+00]
 ...
 [-1.46257e-01  8.32682e-01  5.81137e+00]
 [-1.26488e+00  1.81604e+00  5.44585e+00]
 [-1.43104e+00  6.50741e-03  6.87100e+00]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [11]:
cl_statical2 = statistical_outlier_removal(pcd_second_modify, 20, 3.5) #PointCloud2000Buono
#cl_statical2 = statistical_outlier_removal(pcd_second_modify, 20, 0.5) #PointCloud2000Fallato
#cl_statical2 = statistical_outlier_removal(pcd_second_modify, 60, 0.1) #pointCloudPezzoBuono
#cl_statical2 = statistical_outlier_removal(pcd_second_modify, 80, 0.1) #pointCloudPezzoFallato

Statistical outlier removal
Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [12]:
print(np.asarray(cl_statical2.points))
print(" ")
cl_statical2_colori = np.asarray(cl_statical2.colors) * 255
cl_statical2_colori = cl_statical2_colori.astype(int)
print(cl_statical2_colori)

[[ 1.19864  -1.4703    6.6767  ]
 [-0.652911 -1.15012   6.82994 ]
 [ 1.31762  -1.07569   7.25732 ]
 ...
 [ 1.40265  -1.54295   6.84146 ]
 [ 1.24114  -1.92363   7.00253 ]
 [ 1.33186  -1.50067   6.70713 ]]
 
[[150 166 187]
 [120  76  66]
 [140  94  75]
 ...
 [169 149 150]
 [136 102 104]
 [207 198 197]]


In [150]:
save_point_cloud("pointCloud_Pulita.ply", np.asarray(cl_statical2.points), cl_statical2_colori)

In [24]:
pcd_pulita = visualize_point_cloud("pointCloud_Pulita.ply")

[[ 3.87531e-01  1.41671e+00  5.85540e+00]
 [ 7.42786e-01  3.41862e-01  6.79823e+00]
 [-8.10134e-02 -4.84564e-01  6.86060e+00]
 ...
 [ 8.07342e-01  4.48488e-01  7.00684e+00]
 [ 1.66342e-01 -5.42787e-01  7.11652e+00]
 [-1.43104e+00  6.50741e-03  6.87100e+00]]
